In [189]:
import pandas as pd
import numpy as np
from numpy import vectorize

In [190]:
bikes = pd.read_csv('201602-citibike-tripdata.csv',sep=',',quotechar='"')

In [191]:
bikes.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,461,2/1/2016 00:00:08,2/1/2016 00:07:49,480,W 53 St & 10 Ave,40.766697,-73.990617,524,W 43 St & 6 Ave,40.755273,-73.983169,23292,Subscriber,1966,1
1,297,2/1/2016 00:00:56,2/1/2016 00:05:53,463,9 Ave & W 16 St,40.742065,-74.004432,380,W 4 St & 7 Ave S,40.734011,-74.002939,15329,Subscriber,1977,1
2,280,2/1/2016 00:01:00,2/1/2016 00:05:40,3134,3 Ave & E 62 St,40.763126,-73.965269,3141,1 Ave & E 68 St,40.765005,-73.958185,22927,Subscriber,1987,1
3,662,2/1/2016 00:01:00,2/1/2016 00:12:02,537,Lexington Ave & E 24 St,40.740259,-73.984092,428,E 3 St & 1 Ave,40.724677,-73.987834,20903,Subscriber,1983,2
4,355,2/1/2016 00:01:41,2/1/2016 00:07:36,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,521,8 Ave & W 31 St,40.750967,-73.994442,23228,Subscriber,1978,1


In [192]:
minlat=np.min(bikes['start station latitude']); maxlat=np.max(bikes['start station latitude'])

In [193]:
minlon=np.min(bikes['start station longitude']); maxlon=np.max(bikes['start station longitude'])

In [194]:
print 'center coords: {}, {}'.format(np.mean([minlat,maxlat]),np.mean([minlon,maxlon]))

center coords: 40.73305774, -73.973512775


In [195]:
str_fmt = '''var {name} = L.polyline({latlons}, {{color: {color}, weight:0.5, lineCap:'round'}}).addTo({map});
'''
ll_fmt = 'L.latLng({},{})'
ar_fmt = '[{}]'

In [196]:
ar_fmt.format(','.join([ll_fmt.format(0,0),ll_fmt.format(0,0)]))

'[L.latLng(0,0),L.latLng(0,0)]'

In [197]:
print str_fmt.format(name='line1',latlons='t',color='blue',map='imap')

var line1 = L.polyline(t, {color: blue, weight:0.5, lineCap:'round'}).addTo(imap);



In [198]:
@vectorize
def create_line(slat,slon,elat,elon,name,color,mapname,sstation,estation,duration):
    ll = [ll_fmt.format(slat,slon), ll_fmt.format(elat,elon)]
    latlons = ar_fmt.format(','.join(ll))
    out = str_fmt.format(name=name,latlons=latlons,color=color,map=mapname)
    out += ' {name}.bindPopup("Start Station: {sstation}<br/>End Station: {estation}<br/>Duration: {duration}");'\
                .format(name=name,sstation=sstation,estation=estation,duration=duration)
    return out

In [199]:
np.percentile(bikes['tripduration'],80)

977.0

In [200]:
@vectorize
def getcolor(x):
    if x<=527:
        return '\'#736AFF\''
    else:
        return '\'#7D0552\''

In [201]:
slats = bikes['start station latitude']
slons = bikes['start station longitude']
elats = bikes['end station latitude']
elons = bikes['end station longitude']
names = np.array([ 'line'+str(x) for x in xrange(bikes.shape[0]) ])
colors = getcolor(bikes['tripduration'])
mapnames = np.array([ 'imap' for x in xrange(bikes.shape[0]) ])
sstation = bikes['start station name']
estation = bikes['end station name']
duration = (bikes['tripduration'] // 60).astype('string') +' minutes'

In [202]:
lines = create_line(slats,slons,elats,elons,names,colors,mapnames,sstation,estation,duration)

In [204]:
with open('lines.js','w') as w:
    for l in np.random.choice(lines,10000,replace=False):
        w.write(l+'\n')